# Data analysis: Velib

Author: O. Roustant, INSA Toulouse. February 2022.


We consider the ‘Vélib’ data set, related to the bike sharing system of Paris. The data are loading profiles of the bike stations over one week, collected every hour, from the period Monday 2nd Sept. - Sunday 7th Sept., 2014. The loading profile of a station, or simply loading, is defined as the ratio of number of available bikes divided by the number of bike docks. A loading of 1 means that the station is fully loaded, i.e. all bikes are available. A loading of 0 means that the station is empty, all bikes have been rent.

From the viewpoint of data analysis, the individuals are the stations. The variables are the 168 time steps (hours in the week). The aim is to detect clusters in the data, corresponding to common customer usages. This clustering should then be used to predict the loading profile.

In [ ]:
rm(list = ls())   # erase everything, start from scratch!

# load the data from package funFEM

library(funFEM)
data(velib)
help("velib")

In [ ]:
# data preparation
x <- as.matrix(velib$data)
colnames(x) <- 1:ncol(x)
rownames(x) <- velib$names

n <- nrow(x)
stations <- 1:n 
coord <- velib$position[stations,]

# select exactly 7 days of data (we remove the first 13 dates)
dates <- 14:181
x <- x[stations, dates]
colnames(x) <- 1:length(dates)

In [ ]:
timeTick <- 1 + 24*(0:6)  # vector corresponding to the beginning of days
par(mfrow = c(1, 1))

options(repr.plot.width = 15, repr.plot.height = 6)

plot(x[1, ], col = "blue", type = "l", ylim = c(0, 1), 
     xlab = "Time", ylab = "Loading", main = rownames(x)[1])
abline(v = timeTick, lty = "dotted")


In [ ]:
# From now on, we use numbers instead of station names, 
# in order to simplify printing
# rownames(x) <- 1:nrow(x)

# Descriptive statistics.

Some ideas : 

1. Draw a matrix of plots of size 4*4, corresponding to the first 16 stations. (Do not forget the vertical lines corresponding to days).
2. Draw the boxplot of the variables, sorted in time order. 
What can you say about the distribution of the variables? 
Position, dispersion, symmetry? Can you see a difference between days?
3. Plot the average hourly loading for each day (on a single graph).
Comments? 
4. Plot the stations coordinates on a 2D map (latitude versus longitude). Use the package ggmap (function 'qmplot') to visualize the average loading for a given hour (6h, 12h, 23h) as a color scale.
Comments ?
5. Use a different color for stations which are located on a hill. (Use the basis 'plot' function, and the function 'qmplot' of R package ggmap).
6. Redo questions 1-3 for the subset of stations which are located on a hill. Same questions for those who are not. Comment?

### Loading in a week of the first 16 stations

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 12)

par(mfrow = c(4,4))
for (sta in 1:16){
    plot(x[sta, ], col = "blue", type = "l", ylim = c(0, 1), 
     xlab = "Time", ylab = "Loading", main = paste('No.',sta,' - ',rownames(x)[sta]))
    abline(v = timeTick, lty = "dotted")
}

### Boxplot of hourly loading of each day

In [ ]:
boxplot(x,ylab='loading',xlab='time (hour)')
abline(v = timeTick,col='red')

##### Comment : 
+ Range : 0.1 - 0.8
+ Symmetry : No, more dispersed on the upper partition of the median.
+ Difference between days : Yes, especially between weekdays and weekends. Not much, among weekdays (same with weekends).

### Average hourly loading of each day

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
plot(colMeans(x),type='l',xlab='time (hour)',ylab='loading',main='Average hourly loading')
abline(v = timeTick, lty = "dotted")

### Map of stations, colored by average loading at 6h, 12h, 23h

In [ ]:
library(ggmap)

In [ ]:
sta = 1:nrow(x)

tick6h = 6 + 24*(0:6)
tick12h = 12 + 24*(0:6)
tick23h = 23 + 24*(0:6)
avg6h = rowMeans(x[sta,tick6h])
avg12h = rowMeans(x[sta,tick12h])
avg23h = rowMeans(x[sta,tick23h])

lon = coord[sta,1]
lat = coord[sta,2]
bonus = velib$bonus[sta]
xmap = cbind(lon, lat, avg6h, avg12h, avg23h,bonus)
xmap = as.data.frame(xmap)

In [ ]:
# theme_update(plot.title = element_text(hjust = 0.5))
# options(repr.plot.width = 15, repr.plot.height = 10)
# qmplot(x=lon, y=lat, data=xmap, colour = avg23h, size=I(12), alpha=I(0.7),zoom=13) +
#     geom_point(aes(x = lon, y = lat, colour = avg12h, size = I(8),alpha=I(0.7),stroke=0.5),data = xmap) + 
#     geom_point(aes(x = lon, y = lat, colour = avg6h, size = I(4),alpha=I(0.7)),data = xmap) + 
#     scale_color_gradient2(high = "red") +
#     ggtitle("Map of stations, colored by average loading at 6h,12h,23h")

In [ ]:
theme_update(plot.title = element_text(hjust = 0.5))
options(repr.plot.width = 15, repr.plot.height = 10)
qmplot(x=lon, y=lat, data=xmap, colour = avg6h, size=I(5), alpha=I(0.7),zoom=13) + 
    ggtitle("Map of stations, colored by average loading at 6h")
qmplot(x=lon, y=lat, data=xmap, colour = avg12h, size=I(5), alpha=I(0.7),zoom=13) + 
    ggtitle("Map of stations, colored by average loading at 12h")
qmplot(x=lon, y=lat, data=xmap, colour = avg23h, size=I(5), alpha=I(0.7),zoom=13) + 
    ggtitle("Map of stations, colored by average loading at 23h")

### Hill

In [ ]:
qmplot(x=lon, y=lat, data=xmap, colour = factor(bonus), size=I(5), alpha=I(0.7),zoom=13)

In [ ]:
theme_update(plot.title = element_text(hjust = 0.5))
options(repr.plot.width = 15, repr.plot.height = 10)

qmplot(x=lon, y=lat, data=xmap, colour = factor(bonus), size=I(5), alpha=I(avg6h**2),zoom=13) + scale_color_manual(values = c("red","blue")) +
    ggtitle("Map of stations, colored by average loading at 6h")

qmplot(x=lon, y=lat, data=xmap, colour = factor(bonus), size=I(5), alpha=I(avg12h**2),zoom=13) + scale_color_manual(values = c("red","blue")) +
    ggtitle("Map of stations, colored by average loading at 12h")

qmplot(x=lon, y=lat, data=xmap, colour = factor(bonus), size=I(5), alpha=I(avg23h**2),zoom=13) + scale_color_manual(values = c("red","blue")) +
    ggtitle("Map of stations, colored by average loading at 23h")

### Hill

In [ ]:
x_hill = x[which(bonus==1),]
x_land = x[which(bonus==0),]

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 12)

par(mfrow = c(4,4))
for (sta in 1:16){
    plot(x_hill[sta, ], col = "blue", type = "l", ylim = c(0, 1), 
     xlab = "Time", ylab = "Loading", main = paste('No.',sta,' - ',rownames(x_hill)[sta]))
    abline(v = timeTick, lty = "dotted")
}

In [ ]:
boxplot(x_hill,ylab='loading',xlab='time (hour)')
abline(v = timeTick,col='red')

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
plot(colMeans(x_hill),type='l',xlab='time (hour)',ylab='loading',main='Average hourly loading')
abline(v = timeTick, lty = "dotted")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 12)

par(mfrow = c(4,4))
for (sta in 1:16){
    plot(x_land[sta, ], col = "blue", type = "l", ylim = c(0, 1), 
     xlab = "Time", ylab = "Loading", main = paste('No.',sta,' - ',rownames(x_land)[sta]))
    abline(v = timeTick, lty = "dotted")
}

In [ ]:
boxplot(x_land,ylab='loading',xlab='time (hour)')
abline(v = timeTick,col='red')

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
plot(colMeans(x_land),type='l',xlab='time (hour)',ylab='loading',main='Average hourly loading')
abline(v = timeTick, lty = "dotted")

In [ ]:
plot(colMeans(x_hill),type='l',xlab='time (hour)',ylab='loading',main='Average hourly loading')
lines(colMeans(x_land),col="red")
legend("right",legend=c("Not on hill","On hill"),
       col=c("red", "black"), lty=1:1, cex=0.8)